## Complete Data Analysis for all customer categories

In [1]:
# Importing the required libraries
import pandas as pd
import numpy as np
import networkx as nx
from pyhive import hive
import pydoop.hdfs as hdfs
from pyhive import hive
from impala.dbapi import connect
from impala.util import as_pandas

In [2]:
# Establishing the connection with Hive
host_name = "10.107.6.14"
port = 10000
user = "hdfs"
password = "bigid#01"
database="cx_test_ro_data"
conn = hive.Connection(host=host_name, port=port, username=user, password=password,
                           database=database, auth='CUSTOM')
cursor_1 = conn.cursor()

In [4]:
#Reading table having sale date for vins
cursor_1.execute('select * from complete_data_for_ro_details')
all_data = as_pandas(cursor_1)

In [9]:
print(all_data.shape)
all_data.head(2)

(26487361, 11)


,vin_number,model_name,year_of_data,customer_category,jdp_or_nonjdp,repm_vin,repm_work_type,repm_estmt_amt,repm_bill_total_part_amt,repm_bill_total_labr_amt,repm_bill_total_othr_amt
0,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,RR,0.00,0.00,0.00,0.00
1,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,PS,1595.00,6355.00,4801.11,0.00


In [10]:
# Calculate the total bill 
all_data['total_bill'] = all_data['repm_bill_total_part_amt'] + all_data['repm_bill_total_labr_amt'] + all_data['repm_bill_total_othr_amt']
all_data.head(4)

,vin_number,model_name,year_of_data,customer_category,jdp_or_nonjdp,repm_vin,repm_work_type,repm_estmt_amt,repm_bill_total_part_amt,repm_bill_total_labr_amt,repm_bill_total_othr_amt,total_bill
0,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,RR,0.00,0.00,0.00,0.00,0.00
1,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,PS,1595.00,6355.00,4801.11,0.00,11156.11
2,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,PS,0.00,4833.39,3929.40,0.00,8762.79
3,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,RR,0.00,22326.01,0.00,0.00,22326.01


In [11]:
# Calculating total estimate amt for all VINs
all_data['total_estmt_amt'] = all_data.repm_vin.map(all_data.groupby('repm_vin')['repm_estmt_amt'].sum())
all_data.head(4)

,vin_number,model_name,year_of_data,customer_category,jdp_or_nonjdp,repm_vin,repm_work_type,repm_estmt_amt,repm_bill_total_part_amt,repm_bill_total_labr_amt,repm_bill_total_othr_amt,total_bill,total_estmt_amt
0,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,RR,0.00,0.00,0.00,0.00,0.00,41850.00
1,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,PS,1595.00,6355.00,4801.11,0.00,11156.11,41850.00
2,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,PS,0.00,4833.39,3929.40,0.00,8762.79,41850.00
3,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,RR,0.00,22326.01,0.00,0.00,22326.01,41850.00


In [12]:
# Calculating total bill for all VINs
all_data['total_bill_amt'] = all_data.repm_vin.map(all_data.groupby('repm_vin')['total_bill'].sum())
all_data.head(4)

,vin_number,model_name,year_of_data,customer_category,jdp_or_nonjdp,repm_vin,repm_work_type,repm_estmt_amt,repm_bill_total_part_amt,repm_bill_total_labr_amt,repm_bill_total_othr_amt,total_bill,total_estmt_amt,total_bill_amt
0,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,RR,0.00,0.00,0.00,0.00,0.00,41850.00,266242.36
1,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,PS,1595.00,6355.00,4801.11,0.00,11156.11,41850.00,266242.36
2,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,PS,0.00,4833.39,3929.40,0.00,8762.79,41850.00,266242.36
3,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,RR,0.00,22326.01,0.00,0.00,22326.01,41850.00,266242.36


In [14]:
# Removing duplicates from the above data
clean_data_ro = all_data.drop_duplicates(subset = ['vin_number', 'total_estmt_amt', 'total_bill_amt'])
print(clean_data_ro.shape)
clean_data_ro.head(4)

(1286955, 14)


,vin_number,model_name,year_of_data,customer_category,jdp_or_nonjdp,repm_vin,repm_work_type,repm_estmt_amt,repm_bill_total_part_amt,repm_bill_total_labr_amt,repm_bill_total_othr_amt,total_bill,total_estmt_amt,total_bill_amt
0,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,RR,0.00,0.00,0.00,0.00,0.00,41850.00,266242.36
34,MALCU41ULDM143122,NEW VERNA,2019,Dissatisfied,JDP,MALCU41ULDM143122,AR,0.00,9776.98,20147.32,0.00,29924.30,7000.00,72646.58
43,MALCU41ULDM143118,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143118,PS,0.00,5810.44,5032.97,0.00,10843.41,49810.00,96805.23
65,MALCU41ULDM143091,NEW VERNA,2020,Dissatisfied,JDP,MALCU41ULDM143091,AR,0.00,3296.31,4494.40,0.00,7790.71,2600.00,134233.88


In [15]:
# Calculating the difference between actual bill amt and total estmt amt
clean_data_ro['actual_estmt_diff'] = clean_data_ro['total_bill_amt'] - clean_data_ro['total_estmt_amt']
clean_data_ro.head(3)

# If the difference is in positive then that means that the actual bill was more than the estmt amt

/usr/local/src/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,vin_number,model_name,year_of_data,customer_category,jdp_or_nonjdp,repm_vin,repm_work_type,repm_estmt_amt,repm_bill_total_part_amt,repm_bill_total_labr_amt,repm_bill_total_othr_amt,total_bill,total_estmt_amt,total_bill_amt,actual_estmt_diff
0,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,RR,0.00,0.00,0.00,0.00,0.00,41850.00,266242.36,224392.36
34,MALCU41ULDM143122,NEW VERNA,2019,Dissatisfied,JDP,MALCU41ULDM143122,AR,0.00,9776.98,20147.32,0.00,29924.30,7000.00,72646.58,65646.58
43,MALCU41ULDM143118,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143118,PS,0.00,5810.44,5032.97,0.00,10843.41,49810.00,96805.23,46995.23


## Calculating ro visit count for the VINs

In [16]:
# Reading table for ro visit count
cursor_1.execute('select * from ro_visit_count_all_data')
ro_count = as_pandas(cursor_1)

In [17]:
print(ro_count.shape)
ro_count.head(3)

(6501861, 2)


,repm_vin,count_visit
0,********00507****,2
1,*****1525********,1
2,**MBJ11JV400711**,1


In [19]:
# Merging the ro visit count with the vins 
merged_ro_count = pd.merge(clean_data_ro, ro_count[["repm_vin","count_visit"]], left_on = "vin_number", right_on = 'repm_vin')
print(merged_ro_count.shape)
merged_ro_count.head(3)

(1286955, 17)


,vin_number,model_name,year_of_data,customer_category,jdp_or_nonjdp,repm_vin_x,repm_work_type,repm_estmt_amt,repm_bill_total_part_amt,repm_bill_total_labr_amt,repm_bill_total_othr_amt,total_bill,total_estmt_amt,total_bill_amt,actual_estmt_diff,repm_vin_y,count_visit
0,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,RR,0.00,0.00,0.00,0.00,0.00,41850.00,266242.36,224392.36,MALCU41ULDM143128,35
1,MALCU41ULDM143122,NEW VERNA,2019,Dissatisfied,JDP,MALCU41ULDM143122,AR,0.00,9776.98,20147.32,0.00,29924.30,7000.00,72646.58,65646.58,MALCU41ULDM143122,11
2,MALCU41ULDM143118,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143118,PS,0.00,5810.44,5032.97,0.00,10843.41,49810.00,96805.23,46995.23,MALCU41ULDM143118,22


In [21]:
merged_ro_count = merged_ro_count.rename({'count_visit' : 'RO_visit_count'}, axis=1)
merged_ro_count.head(2)

,vin_number,model_name,year_of_data,customer_category,jdp_or_nonjdp,repm_vin_x,repm_work_type,repm_estmt_amt,repm_bill_total_part_amt,repm_bill_total_labr_amt,repm_bill_total_othr_amt,total_bill,total_estmt_amt,total_bill_amt,actual_estmt_diff,repm_vin_y,RO_visit_count
0,MALCU41ULDM143128,New Verna,2019,Dissatisfied,JDP,MALCU41ULDM143128,RR,0.00,0.00,0.00,0.00,0.00,41850.00,266242.36,224392.36,MALCU41ULDM143128,35
1,MALCU41ULDM143122,NEW VERNA,2019,Dissatisfied,JDP,MALCU41ULDM143122,AR,0.00,9776.98,20147.32,0.00,29924.30,7000.00,72646.58,65646.58,MALCU41ULDM143122,11


### Calculating referral count and the retail count for the vins

In [22]:
# Reading table for referral count
cursor_1.execute('select * from referral_count_for_all_data')
rfrl_count = as_pandas(cursor_1)

In [23]:
print(rfrl_count.shape)
rfrl_count.head(3)

(585448, 2)


,refl_vin,total_referral_count
0,...........544489,2
1,...........566698,2
2,..........E233621,2


In [24]:
# Merging the referral count with the vins 
merged_rfrl_count = pd.merge(merged_ro_count, rfrl_count[["refl_vin","total_referral_count"]], left_on = "vin_number", right_on = 'refl_vin')
print(merged_rfrl_count.shape)
merged_rfrl_count.head(3)

(200288, 19)


,vin_number,model_name,year_of_data,customer_category,jdp_or_nonjdp,repm_vin_x,repm_work_type,repm_estmt_amt,repm_bill_total_part_amt,repm_bill_total_labr_amt,repm_bill_total_othr_amt,total_bill,total_estmt_amt,total_bill_amt,actual_estmt_diff,repm_vin_y,RO_visit_count,refl_vin,total_referral_count
0,MALCU41ULDM142839,New Verna,2019,Dissatisfied,Non JDP,MALCU41ULDM142839,PS,13000.00,7881.08,6334.22,0.00,14215.30,33950.00,382163.91,348213.91,MALCU41ULDM142839,17,MALCU41ULDM142839,2
1,MALCU41ULDM142787,New Verna,2019,Satisfied,Non JDP,MALCU41ULDM142787,OC,0.00,0.00,0.00,0.00,0.00,17200.00,120875.33,103675.33,MALCU41ULDM142787,17,MALCU41ULDM142787,2
2,MALCU41ULDM142699,NEW VERNA,2020,Dissatisfied,JDP,MALCU41ULDM142699,FS,2500.00,2359.86,157.30,0.00,2517.16,15500.00,35940.79,20440.79,MALCU41ULDM142699,7,MALCU41ULDM142699,2


### Calculating retail count for all vins

In [25]:
# Reading table for retail count
cursor_1.execute('select * from retail_count')
retail_count = as_pandas(cursor_1)

In [26]:
print(retail_count.shape)
retail_count.head(3)

(136270, 2)


,refl_vin,_c1
0,00000000000000132,1
1,00000000000000926,1
2,00000000000002016,1


In [27]:
retail_count = retail_count.rename({'_c1' : 'Total_retail_count'}, axis=1)
retail_count.head(3)

,refl_vin,Total_retail_count
0,00000000000000132,1
1,00000000000000926,1
2,00000000000002016,1


In [29]:
final_df = pd.merge(merged_rfrl_count, retail_count[['refl_vin', 'Total_retail_count']], how = 'left', left_on = "vin_number", right_on = 'refl_vin')
print(final_df.shape)
final_df.head(3)

(200288, 21)


,vin_number,model_name,year_of_data,customer_category,jdp_or_nonjdp,repm_vin_x,repm_work_type,repm_estmt_amt,repm_bill_total_part_amt,repm_bill_total_labr_amt,...,total_bill,total_estmt_amt,total_bill_amt,actual_estmt_diff,repm_vin_y,RO_visit_count,refl_vin_x,total_referral_count,refl_vin_y,Total_retail_count
0,MALCU41ULDM142839,New Verna,2019,Dissatisfied,Non JDP,MALCU41ULDM142839,PS,13000.00,7881.08,6334.22,...,14215.30,33950.00,382163.91,348213.91,MALCU41ULDM142839,17,MALCU41ULDM142839,2,MALCU41ULDM142839,1.0
1,MALCU41ULDM142787,New Verna,2019,Satisfied,Non JDP,MALCU41ULDM142787,OC,0.00,0.00,0.00,...,0.00,17200.00,120875.33,103675.33,MALCU41ULDM142787,17,MALCU41ULDM142787,2,NaN,NaN
2,MALCU41ULDM142699,NEW VERNA,2020,Dissatisfied,JDP,MALCU41ULDM142699,FS,2500.00,2359.86,157.30,...,2517.16,15500.00,35940.79,20440.79,MALCU41ULDM142699,7,MALCU41ULDM142699,2,NaN,NaN


In [30]:
# Dropping unnecessary columns from the data frame
cleaned_df = final_df[['vin_number', 'total_bill', 'total_estmt_amt', 'total_bill_amt', 'actual_estmt_diff', 'RO_visit_count', 'total_referral_count', 'Total_retail_count']]
print(cleaned_df.shape)
cleaned_df.head(3)

(200288, 8)


,vin_number,total_bill,total_estmt_amt,total_bill_amt,actual_estmt_diff,RO_visit_count,total_referral_count,Total_retail_count
0,MALCU41ULDM142839,14215.30,33950.00,382163.91,348213.91,17,2,1.0
1,MALCU41ULDM142787,0.00,17200.00,120875.33,103675.33,17,2,NaN
2,MALCU41ULDM142699,2517.16,15500.00,35940.79,20440.79,7,2,NaN


### Calculating the vehcile age from the sale date

In [31]:
#Reading table having sale date for vins
cursor_1.execute('select * from sale_date_for_all_data')
sale_date = as_pandas(cursor_1)

In [32]:
print(sale_date.shape)
sale_date.head(3)

(2200381, 7)


,vin_number,model_name,year_of_data,customer_category,jdp_or_nonjdp,vinm_vin,vinm_sale_date
0,MALA851CLEM072408,GRAND i10,2020,Dissatisfied,JDP,MALA851CLEM072408,20140305
1,MALA851CLEM077424,Grand i10,2019,Satisfied,JDP,MALA851CLEM077424,20140313
2,MALAN51CLBM852454,i10,2019,Dissatisfied,Non JDP,MALAN51CLBM852454,20110412


In [33]:
sale_date = sale_date.replace('null', np.nan)

In [34]:
# Converting the sale date into timestamp format
sale_date['vinm_sale_date'] = pd.to_datetime(sale_date['vinm_sale_date'], infer_datetime_format=True)

In [35]:
sale_date.head(3)

,vin_number,model_name,year_of_data,customer_category,jdp_or_nonjdp,vinm_vin,vinm_sale_date
0,MALA851CLEM072408,GRAND i10,2020,Dissatisfied,JDP,MALA851CLEM072408,2014-03-05
1,MALA851CLEM077424,Grand i10,2019,Satisfied,JDP,MALA851CLEM077424,2014-03-13
2,MALAN51CLBM852454,i10,2019,Dissatisfied,Non JDP,MALAN51CLBM852454,2011-04-12


In [36]:
sale_date['end_date'] = '2020-12-31'

In [37]:
sale_date['end_date'] = pd.to_datetime(sale_date['end_date'], infer_datetime_format=True)

In [38]:
# Creating a new column to calculate the vehicle age
sale_date['vehicle_age'] =  sale_date['end_date'] - sale_date['vinm_sale_date']
sale_date['vehicle_age'] =  sale_date['vehicle_age']/np.timedelta64(1,'Y')

In [39]:
# Rounding off the vehicle age
sale_date['vehicle_age'] = sale_date['vehicle_age'].apply(np.ceil)
sale_date['vehicle_age'].value_counts()

 5.0      627513
 6.0      484118
 4.0      461212
 3.0      269973
 7.0      110290
 2.0       89543
 8.0       43214
 1.0       31603
 9.0       31036
 10.0      23478
 11.0      17548
 12.0      10744
 13.0         37
 14.0         25
 15.0         14
 16.0         11
 17.0          6
-180.0         2
 18.0          2
 20.0          2
 19.0          1
Name: vehicle_age, dtype: int64

In [40]:
sale_date['vehicle_age_category'] = sale_date['vehicle_age']
sale_date.head(3)

,vin_number,model_name,year_of_data,customer_category,jdp_or_nonjdp,vinm_vin,vinm_sale_date,end_date,vehicle_age,vehicle_age_category
0,MALA851CLEM072408,GRAND i10,2020,Dissatisfied,JDP,MALA851CLEM072408,2014-03-05,2020-12-31,7.0,7.0
1,MALA851CLEM077424,Grand i10,2019,Satisfied,JDP,MALA851CLEM077424,2014-03-13,2020-12-31,7.0,7.0
2,MALAN51CLBM852454,i10,2019,Dissatisfied,Non JDP,MALAN51CLBM852454,2011-04-12,2020-12-31,10.0,10.0


In [41]:
sale_date['vehicle_age_category'] = sale_date['vehicle_age_category'].astype(str)
sale_date.dtypes

vin_number                      object
model_name                      object
year_of_data                    object
customer_category               object
jdp_or_nonjdp                   object
vinm_vin                        object
vinm_sale_date          datetime64[ns]
end_date                datetime64[ns]
vehicle_age                    float64
vehicle_age_category            object
dtype: object

In [42]:
sale_date['vehicle_age_category'] = sale_date['vehicle_age_category'].replace(['1.0'], '1 Year')
sale_date['vehicle_age_category'] = sale_date['vehicle_age_category'].replace(['2.0'], '2 Years')
sale_date['vehicle_age_category'] = sale_date['vehicle_age_category'].replace(['3.0'], '3 Years')
sale_date['vehicle_age_category'] = sale_date['vehicle_age_category'].replace(['4.0'], '4 Years')
sale_date['vehicle_age_category'] = sale_date['vehicle_age_category'].replace(['5.0'], '5 Years')
sale_date['vehicle_age_category'] = sale_date['vehicle_age_category'].replace(['6.0', '7.0', '8.0', '9.0', '10.0'], '> 5 Years')
sale_date['vehicle_age_category'] = sale_date['vehicle_age_category'].replace(['11.0', '12.0', '13.0','14.0','15.0', '16.0', '17.0', '18.0', '19.0', '20.0'], '> 10 Years')

In [43]:
# Merging the vehicle age with the cleaned df
final_df_with_all_columns = pd.merge(cleaned_df, sale_date[['vin_number', 'vehicle_age','vehicle_age_category']], how = 'left', on = "vin_number")
print(final_df_with_all_columns.shape)
final_df_with_all_columns.head(3)

(388967, 10)


,vin_number,total_bill,total_estmt_amt,total_bill_amt,actual_estmt_diff,RO_visit_count,total_referral_count,Total_retail_count,vehicle_age,vehicle_age_category
0,MALCU41ULDM142839,14215.30,33950.00,382163.91,348213.91,17,2,1.0,7.0,> 5 Years
1,MALCU41ULDM142787,0.00,17200.00,120875.33,103675.33,17,2,NaN,8.0,> 5 Years
2,MALCU41ULDM142699,2517.16,15500.00,35940.79,20440.79,7,2,NaN,8.0,> 5 Years


In [44]:
# Saving as csv
final_df_with_all_columns.to_csv("data_all_cust_cat_all_cols.csv", encoding = "utf-8")